# Prepare Notre Dame data

In [26]:
%matplotlib widget
import numpy
import utils

## Load data into Python

In [27]:
def read_cam(file_stream):
    line = file_stream.readline().split()
    cam = {}
    # Intrinsics.
    cam["focal"] = float(line[0])
    cam["k1"] = float(line[1])
    cam["k2"] = float(line[2])

    # Extrinsics.
    cam["R"] = numpy.zeros((3, 3))
    for i in range(3):
        line = file_stream.readline().split()
        cam["R"][i] = numpy.array([float(val) for val in line])
    
    line = file_stream.readline().split()
    cam["t"] = numpy.array([float(val) for val in line])

    return cam


def read_point(file_stream):
    line = file_stream.readline().split()
    # 3D coordinates of the point.
    X = numpy.array([float(val) for val in line])

    # We don't care about the color, so we just read the line and skip it.
    line = file_stream.readline().split()
    # If we were to care about the color, we would do this:
    # pt["color"] = numpy.array([int(val) for val in line])
    
    # View list.
    line = file_stream.readline().split()
    num_cams = int(line[0])
    cam_idxs = []
    cam_coords = []
    for i in range(num_cams):
        # Camera index.
        cam_idx = int(line[1 + 4*i])
        cam_idxs.append(cam_idx)
        
        # 2D coordinates of the point in the camera.
        x = float(line[3 + 4*i])
        y = float(line[4 + 4*i])
        cam_coords.append(numpy.array([x, y]))

    return X, cam_idxs, cam_coords

def undistort(coords_, cam):
    norm_coords_ = numpy.linalg.norm(coords_)
    r_ = norm_coords_ / cam["focal"]
    r_old = r_
    r_new = r_ / (1 + cam["k1"] * r_**2 + cam["k2"] * r_**4)
    epsilon = 1e-8
    while abs(r_new - r_old) > epsilon:
        r_old = r_new
        r_new = r_ / (1 + cam["k1"] * r_old**2 + cam["k2"] * r_old**4)

    return coords_ / norm_coords_ * r_new


with open("notre_dame/notredame.out", "r") as f:
    line = ["#"]
    while line[0] == "#":
        line = f.readline().split()
    num_cameras = int(line[0])
    num_points = int(line[1])
    cams = []
    for _ in range(num_cameras):
        cams.append(read_cam(f))

    points_3D = numpy.empty((num_points, 3))
    pt_idx_to_cam_idxs = []
    pt_idx_to_coords_in_cams = []
    for i in range(num_points):
        X, cam_idxs, cam_coords_ = read_point(f)
        points_3D[i] = X
        pt_idx_to_cam_idxs.append(cam_idxs)
        
        # Undistort the 2D coordinates.
        cam_coords = numpy.empty((len(cam_idxs), 2))
        for j, image_pt in enumerate(cam_coords_):
            cam_idx = cam_idxs[j]
            cam_coords[j] = undistort(image_pt, cams[cam_idx])
        
        pt_idx_to_coords_in_cams.append(cam_coords)

def invert_map(pt_idx_to_cam_idxs):
    cam_idx_to_pt_idxs = {}
    for pt_idx, cam_idxs in enumerate(pt_idx_to_cam_idxs):
        for cam_idx in cam_idxs:
            if cam_idx not in cam_idx_to_pt_idxs.keys():
                cam_idx_to_pt_idxs[cam_idx] = []
            cam_idx_to_pt_idxs[cam_idx].append(pt_idx)
    return cam_idx_to_pt_idxs

cam_idx_to_pt_idxs = invert_map(pt_idx_to_cam_idxs)

## Save in standard format

In [ ]:
# Reformat cameras.
cameras = []
for camera in cams:
    K = numpy.array([[camera["focal"], 0, 0], [0, camera["focal"], 0], [0, 0, 1]])
    R = camera["R"]
    t = camera["t"]
    cameras.append(numpy.hstack((K, R, t[:, None])))

# Save cameras.
utils.save_cameras("notre_dame/cameras.bin", cameras)

# Reformat measurements.
measurements = []
for pt_idx, cam_coords in enumerate(pt_idx_to_coords_in_cams):
    measurement = numpy.full((len(cameras), 2), -1.0)
    for i, cam_idx in enumerate(pt_idx_to_cam_idxs[pt_idx]):
        measurement[cam_idx] = pt_idx_to_coords_in_cams[pt_idx][i]
        
    measurements.append(measurement)

# Save measurements.
utils.save_measurements("notre_dame/measurements.bin", measurements)

# Save points.
utils.save_points("notre_dame/points.bin", points_3D)

In [25]:
pt_idx_to_coords_in_cams[0]

array([[ 5.87250534e-02, -2.27706376e-02],
       [-1.45849579e-02, -1.27388325e-01],
       [ 5.50705308e-02, -8.24213645e-02],
       [ 5.55158991e-02, -7.30904309e-02],
       [ 3.66892529e-02, -6.71784419e-02],
       [ 5.68729525e-02, -3.84532004e-02],
       [ 3.28631462e-02, -4.81887285e-02],
       [ 2.01723719e-01, -8.68701600e-02],
       [ 1.03012752e-01,  5.37079129e-02],
       [ 9.97635887e-02,  4.33033513e-01],
       [-3.23205919e-03, -2.25559851e-01],
       [ 1.62089230e-02, -5.52549718e-02],
       [ 1.25411349e-02, -1.13895758e-01],
       [-2.80937718e-02, -1.51009050e-01],
       [ 7.82887322e-02, -3.03868428e-01],
       [ 9.98227414e-02, -1.53125236e-01],
       [ 6.17401548e-02,  8.15438917e-02],
       [ 5.71779965e-02, -1.83356768e-01],
       [ 4.70293484e-02, -1.28000135e-01],
       [-2.85488449e-02, -1.52408382e-01],
       [ 3.99020064e-02, -8.01390753e-02],
       [-6.34380368e-03,  1.00088149e-01],
       [ 1.22310610e-01,  7.92386903e-02],
       [ 2.

In [24]:
len(pt_idx_to_cam_idxs[0])

167